In [1]:
import pandas as pd
import numpy as np
import os
import csv

In [2]:
#Event='AscendEx'
Event='Upbit'
raw_path='../Dataset/'+Event+'/all_data_token/'
address_path='../Dataset/'+Event+'/dataset/'

In [3]:
with open('../Code_Results/Tx_Velocity_Results/'+Event+'_Tx_Internal.csv','w') as f:

    csv_write = csv.writer(f)
    csv_head = ["address","Tx_Velocity"]
    csv_write.writerow(csv_head)

output = pd.read_csv('../Code_Results/Tx_Velocity_Results/'+Event+'_Tx_Internal.csv')

In [4]:
#Step1：筛选出之发生了ETH交易的洗钱账户（source+layering）
#方法2：all_erc20_address和all_normal_address的交集，再用all_normal_address减掉交集
all_address=pd.read_csv(address_path+"all-address.csv")
all_erc20_address=pd.read_csv(address_path+"all-erc20-address.csv")
all_normal_address=pd.read_csv(address_path+"all-normal-address.csv")
address=all_address['address']
erc20_address=all_erc20_address['address']
normal_address=all_normal_address['address']
ETH_address=set(address)-set(erc20_address)
temp=set(erc20_address)&set(normal_address)
ETH_address=list(set(normal_address)-temp)
#再筛选出来label是heist的账户
all_ETH_address=all_normal_address[all_normal_address['address'].isin(ETH_address)]
# ETH_heist_address=list(all_ETH_address.loc[(all_ETH_address['label']=='heist')&(all_ETH_address['name_tag']!='ml_transit_9')]['address'])
ETH_heist_address=list(all_ETH_address.loc[(all_ETH_address['label']=='heist')&(all_ETH_address['name_tag']!='ml_transit_0')]['address'])

In [5]:
output['address']=ETH_heist_address

In [6]:
#Step2：筛选出ETH账户的交易
all_normal_tx=pd.read_csv(address_path+"all-normal-tx.csv")
ETH_tx=all_normal_tx[all_normal_tx['from'].isin(list(ETH_heist_address)) | all_normal_tx['to'].isin(list(ETH_heist_address))]
ETH_tx.sort_values(by="timeStamp" , inplace=True, ascending=True) #按照交易时间升序排序

<ipython-input-6-11bd2c37dd0a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ETH_tx.sort_values(by="timeStamp" , inplace=True, ascending=True) #按照交易时间升序排序


In [52]:
#ETH_heist_address=['0xe7ef8592e64df04891e1e3bbf3013970b5a54f43']

In [7]:
#Step3：统计每个账户的交易速度平均值
import math
for i in range(0,len(ETH_heist_address)):

    Del_T=[]
    print('%d/%d'%(i,len(ETH_heist_address)),end="\r")
#     record=ETH_tx.loc[ETH_tx["isError"]!=1]
    record=ETH_tx
    Ac=ETH_heist_address[i]
    Ac_Tx=record.loc[(record['from']==Ac)|(record['to']==Ac)]
    for j in range(0,len(Ac_Tx)-1):
        Tx=Ac_Tx.iloc[j]
        if Tx['from']==Ac:
            Next_Tx=Ac_Tx.iloc[j+1]
            if Next_Tx['to']==Ac:
                if Next_Tx['timeStamp']-Tx['timeStamp']<604800:
                    Del_T.append(Next_Tx['timeStamp']-Tx['timeStamp'])
        if Tx['to']==Ac:
            Next_Tx=Ac_Tx.iloc[j+1]
            if Next_Tx['from']==Ac:
                if Next_Tx['timeStamp']-Tx['timeStamp']<604800:
                    Del_T.append(Next_Tx['timeStamp']-Tx['timeStamp'])
    if len(Del_T) !=0:
        Avg_Internal=sum(Del_T)/len(Del_T)
        output['Tx_Velocity'][i]=Avg_Internal  
    if len(Del_T) ==0:
        output['Tx_Velocity'][i]=0

In [10]:
output

,address,Tx_Velocity
0,0xe7ef8592e64df04891e1e3bbf3013970b5a54f43,0.196162
1,0x7916ed1b5accf672e277fc06b1538a21a30bfbee,3.44127
2,0x170dedd533c37facd60ee18b51d7bcde274a0d1f,5.14657
3,0x16ffc8a17b60398e4fcdc8c111e56373a60d8db1,58.4016
4,0x6c134879dab2103aa5b55e90a16eaf96d0f31a2c,6.01158
...,...,...
9963,0x466f1a5791fb1c56e8fe45b834d8292a5bb02012,0
9964,0x79078fb3922d85fa6c6c84decdc525c2f400c0bd,0
9965,0xfe5dc2deb44a49e74e0a1dc8ae5b38feedeaf27d,0
9966,0x94fcaae0ad1f517dd99c02af8f2627495315b55c,0


In [9]:
output['Tx_Velocity']=output['Tx_Velocity']/(60*60)#交易的间隔为小时

In [11]:
output=output.loc[output['Tx_Velocity']!=0]
output.to_csv('../Code_Results/Tx_Velocity_Results/'+Event+'_Tx_Internal.csv', index=False)

In [73]:
output['Tx_Velocity'].mean()                                                        

5.669499545647756

In [74]:
max(output['Tx_Velocity'])

166.64499999999998

In [75]:
min(output['Tx_Velocity'])

9.25925925925926e-05

In [76]:
output['Tx_Velocity'].mode()[0]

0.003333333333333333

In [78]:
output['Tx_Velocity'].median()

0.09555555555555556